# Imports

In [ ]:
import sys
sys.path.append('/content/dataset2024')

In [ ]:
!git clone https://github.com/lm-kbc/dataset2024.git

Cloning into 'dataset2024'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 239 (delta 30), reused 32 (delta 27), pack-reused 201
Receiving objects: 100% (239/239), 426.83 KiB | 5.47 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [ ]:
!pip install pandas regex

In [ ]:
import regex, ast, json, requests
import pandas as pd

In [ ]:
### Paths ###
data_path = '/content/dataset2024/data/'

# Extract only subject entity ids per relation
This is currently set to the validation file. Make sure to change the path to use with test or train.

In [ ]:
file_name = 'val.jsonl'
file_path = data_path + file_name
jsonObj = pd.read_json(path_or_buf=file_path, lines=True)

In [ ]:
jsonObj

,SubjectEntityID,SubjectEntity,ObjectEntitiesID,ObjectEntities,Relation
0,Q38215093,honorary doctor of Stockholm University,"[Q34253, Q49103066, Q58023631, Q63827235, Q498...","[Linus Torvalds, Eva Hedlund, Achim Trebst, Ew...",awardWonBy
1,Q691892,Grammy Award for Best Rock Album,"[Q752268, Q606226, Q935369, Q819127, Q907578, ...","[Echoes, Silence, Patience & Grace, The Black ...",awardWonBy
2,Q63072513,honorary doctor of the Yale University,"[Q61247, Q55430, Q43179, Q47667, Q49128, Q3703...","[Jan Assmann, Zhang Yimou, Ellen Johnson Sirle...",awardWonBy
3,Q93956,Order of the Aztec Eagle,"[Q57555, Q43274, Q47854, Q34189, Q40722, Q4093...","[Sabah Al-Ahmad Al-Jaber Al-Sabah, Charles III...",awardWonBy
4,Q18748042,AAAI Fellow,"[Q51781, Q62901, Q62843, Q18749034, Q18619413,...","[Katia Sycara, Sebastian Thrun, Bob Kahn, Mich...",awardWonBy
...,...,...,...,...,...
373,Q26963179,Superstar Duets,[16],[16],seriesHasNumberOfEpisodes
374,Q3752550,Amor a mil,[124],[124],seriesHasNumberOfEpisodes
375,Q906424,Bolek and Lolek,[183],[183],seriesHasNumberOfEpisodes
376,Q1549157,House of Lies,[58],[58],seriesHasNumberOfEpisodes


# Pretty print ids to use in sparql queries
Use this to quickly print out all entity ids per type.

In [ ]:
wd_entities = jsonObj.groupby("Relation", group_keys=True)['SubjectEntityID'].apply(lambda x: "wd:"+ x)
listed_entities = wd_entities.groupby('Relation').apply(list).drop_duplicates()

In [ ]:
relation_order = jsonObj['Relation'].drop_duplicates().to_list()

In [ ]:
for idx, l in enumerate(listed_entities):
  sparql_ready = ' '.join(ent for ent in l)
  print(relation_order[idx] + ': ' + sparql_ready)

awardWonBy: wd:Q38215093 wd:Q691892 wd:Q63072513 wd:Q93956 wd:Q18748042 wd:Q48786 wd:Q17144 wd:Q185667 wd:Q1479435 wd:Q107183707
companyTradesAtStockExchange: wd:Q854867 wd:Q7277589 wd:Q16123125 wd:Q907435 wd:Q54963024 wd:Q5199353 wd:Q864407 wd:Q21178605 wd:Q12035980 wd:Q221062 wd:Q21592182 wd:Q11188787 wd:Q300924 wd:Q892432 wd:Q15955299 wd:Q77877976 wd:Q2345377 wd:Q4633317 wd:Q1997989 wd:Q6897168 wd:Q2104551 wd:Q308952 wd:Q17049674 wd:Q17025355 wd:Q15731340 wd:Q3700394 wd:Q1101323 wd:Q114769734 wd:Q680683 wd:Q1473166 wd:Q477459 wd:Q2767170 wd:Q122549 wd:Q1188938 wd:Q67808185 wd:Q409907 wd:Q26989 wd:Q5023914 wd:Q864194 wd:Q642189 wd:Q1497106 wd:Q175957 wd:Q730179 wd:Q2912754 wd:Q4638513 wd:Q17040878 wd:Q261288 wd:Q52408950 wd:Q48755137 wd:Q67810851 wd:Q9295114 wd:Q662095 wd:Q1509011 wd:Q6671228 wd:Q483915 wd:Q4854129 wd:Q5151892 wd:Q12063032 wd:Q920037 wd:Q17063326 wd:Q55229815 wd:Q30281437 wd:Q4827537 wd:Q910379 wd:Q102286942 wd:Q11316949 wd:Q75203229 wd:Q10874650 wd:Q15272375 wd:Q141

# SPARQL Queries - Utils


In [ ]:
awards_query = """
SELECT ?entity ?entityLabel ?additionalLabel ?entityDescription ?entityAltLabel ?article
WHERE {{
  VALUES ?entity {{{0}}}
  OPTIONAL{{?entity wdt:P571 ?additional.}}
  OPTIONAL{{   ?article schema:about ?entity .
              ?article schema:isPartOf <https://en.wikipedia.org/>.
}}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en".}}
}}
"""

company_query = """
SELECT ?entity ?entityLabel ?additionalLabel ?entityDescription ?entityAltLabel ?article
WHERE {{
  VALUES ?entity {{{0}}}
  OPTIONAL{{?entity wdt:P1454 ?additional.}}
  OPTIONAL{{   ?article schema:about ?entity .
              ?article schema:isPartOf <https://en.wikipedia.org/>.
}}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en".}}
}}
"""

country_query = """
SELECT ?entity ?entityLabel (GROUP_CONCAT(DISTINCT ?addInfoLabel; SEPARATOR="; ") AS ?additionalLabel) ?entityDescription ?entityAltLabel ?article
WHERE {{
  VALUES ?entity {{{0}}}
  OPTIONAL{{?entity wdt:P31 ?addInfo.}}
  OPTIONAL{{   ?article schema:about ?entity .
              ?article schema:isPartOf <https://en.wikipedia.org/>.
}}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en".
                           ?addInfo rdfs:label ?addInfoLabel.
                           ?entity rdfs:label ?entityLabel.
                           ?entity skos:altLabel ?entityAltLabel.
                           ?entity schema:description ?entityDescription.
                           }}
}}

GROUP BY ?entity ?entityLabel ?entityDescription ?entityAltLabel ?article
ORDER BY ?entityLabel
"""

person_query = """
SELECT ?entity ?entityLabel ?additionalLabel ?entityDescription ?entityAltLabel ?article
WHERE {{
  VALUES ?entity {{{0}}}
  OPTIONAL{{?entity wdt:P570 ?additional.}}
  OPTIONAL{{   ?article schema:about ?entity .
              ?article schema:isPartOf <https://en.wikipedia.org/>.
}}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en".}}
}}
"""

series_query = """
SELECT ?entity ?entityLabel ?additionalLabel ?entityDescription ?entityAltLabel ?article
WHERE {{
  VALUES ?entity {{{0}}}
  OPTIONAL{{?entity wdt:P2437 ?additional.}}
  OPTIONAL{{   ?article schema:about ?entity .
              ?article schema:isPartOf <https://en.wikipedia.org/>.
}}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en".}}
}}
"""

In [ ]:
sparql_queries = {
    'awardWonBy': awards_query,
    'companyTradesAtStockExchange': company_query,
    'countryLandBordersCountry': country_query,
    'personHasCityOfDeath': person_query,
    'seriesHasNumberOfEpisodes': series_query
}

In [ ]:
wikipedia_request = """https://en.wikipedia.org/w/api.php?action=query&explaintext=&exsectionformat=plain&prop=extracts&redirects=&titles={0}&format=json"""

In [ ]:
def get_wikipedia_extract(url):
  split_url = url.split("org/wiki/")
  page_name = split_url[1]

  request_url = wikipedia_request.format(page_name)
  json_data = requests.get(request_url).json()

  keys = list(json_data['query']['pages'].keys())
  print(keys)
  return json_data['query']['pages'][keys[0]]['extract']

In [ ]:
def get_result(json_data, var_name):
  result_dict = json_data['results']['bindings'][0]
  if var_name in result_dict.keys():
    return result_dict[var_name]['value']
  else:
    return pd.NA

In [ ]:
# function to get additional data for one (1) entity
def get_additional_data(entity_id, relation_type):

  url = url = 'https://query.wikidata.org/sparql'

  query = sparql_queries[relation_type]
  formatted_query = query.format('wd:' + entity_id)
  response = requests.get(url, params = {'format': 'json', 'query': formatted_query})
  if response.status_code == 200:
    try:
      sparql_data = response.json()

      wiki_extract = pd.NA
      wiki_name = get_result(sparql_data, 'article')
      if not pd.isna(wiki_name):
        wiki_extract = get_wikipedia_extract(wiki_name)
        # extract is truncated to its first section only
        wiki_extract = wiki_extract.split('\n\n\n')[0]
        wiki_extract = wiki_extract.split('\n\n')[0] if len(wiki_extract) > 25000 else wiki_extract

      return {
            'additionalData':  get_result(sparql_data, 'additionalLabel'),
            'entityDescription': get_result(sparql_data, 'entityDescription'),
            'alternativeLabels': get_result(sparql_data, 'entityAltLabel'),
            'wikipediaExtract': wiki_extract
        }
    except ValueError:
      return {
          'additionalData': pd.NA,
          'entityDescription': pd.NA,
          'alternativeLabels': pd.NA,
          'wikipediaExtract': pd.NA
      }
  else:
    # recurse - this helps us get away from the cases where the sparql endpoint refuses the request
    return get_additional_data(entity_id, relation_type)

# Add additional information to data


In [ ]:
additional_data = jsonObj.apply(lambda row: get_additional_data(row.SubjectEntityID, row.Relation), axis='columns', result_type='expand')
full_data = pd.concat([jsonObj, additional_data], axis='columns')

In [ ]:
full_data

,SubjectEntityID,SubjectEntity,ObjectEntitiesID,ObjectEntities,Relation,additionalData,entityDescription,alternativeLabels,wikipediaExtract
0,Q38215093,honorary doctor of Stockholm University,"[Q34253, Q49103066, Q58023631, Q63827235, Q498...","[Linus Torvalds, Eva Hedlund, Achim Trebst, Ew...",awardWonBy,<NA>,<NA>,<NA>,<NA>
1,Q691892,Grammy Award for Best Rock Album,"[Q752268, Q606226, Q935369, Q819127, Q907578, ...","[Echoes, Silence, Patience & Grace, The Black ...",awardWonBy,1995-01-01T00:00:00Z,award,<NA>,The Grammy Award for Best Rock Album is an awa...
2,Q63072513,honorary doctor of the Yale University,"[Q61247, Q55430, Q43179, Q47667, Q49128, Q3703...","[Jan Assmann, Zhang Yimou, Ellen Johnson Sirle...",awardWonBy,<NA>,<NA>,<NA>,<NA>
3,Q93956,Order of the Aztec Eagle,"[Q57555, Q43274, Q47854, Q34189, Q40722, Q4093...","[Sabah Al-Ahmad Al-Jaber Al-Sabah, Charles III...",awardWonBy,1933-12-29T00:00:00Z,Mexican order and is the highest decoration aw...,<NA>,The Mexican Order of the Aztec Eagle (Spanish:...
4,Q18748042,AAAI Fellow,"[Q51781, Q62901, Q62843, Q18749034, Q18619413,...","[Katia Sycara, Sebastian Thrun, Bob Kahn, Mich...",awardWonBy,<NA>,award,"AAAI Fellows, AAAI Fellowship, FAAAI, Fellow o...",Fellowship of the Association for the Advancem...
...,...,...,...,...,...,...,...,...,...
373,Q26963179,Superstar Duets,[16],[16],seriesHasNumberOfEpisodes,1,television series,NaN,Superstar Duets is a 2016 Philippine televisio...
374,Q3752550,Amor a mil,[124],[124],seriesHasNumberOfEpisodes,<NA>,television series,NaN,Amor a mil (Love to a Thousand) is a Colombian...
375,Q906424,Bolek and Lolek,[183],[183],seriesHasNumberOfEpisodes,12,television series,NaN,"Bolek and Lolek (Polish: Bolek i Lolek), also ..."
376,Q1549157,House of Lies,[58],[58],seriesHasNumberOfEpisodes,5,television series,NaN,House of Lies is an American comedy-drama tele...


In [ ]:
# convert df to json lines file
# If using colab: remember to download your files before disconnecting
full_data.to_json('extra_data_' + file_name, orient='records', lines=True)